In [1]:
import pandas as pd
from sqlalchemy import create_engine, Column, String, Integer, Float, Date, ForeignKey
from sqlalchemy.orm import declarative_base
import os

In [2]:
# Initialize Base for ORM
Base = declarative_base()

# Define the tables

class Payer(Base):
    __tablename__ = 'payers'
    Id = Column(String, primary_key=True)
    NAME = Column(String)
    ADDRESS = Column(String)
    CITY = Column(String)
    STATE_HEADQUARTERED = Column(String)
    ZIP = Column(String)
    PHONE = Column(String)


class Encounter(Base):
    __tablename__ = 'encounters'
    Id = Column(String, primary_key=True)
    START = Column(Date)
    STOP = Column(Date)
    PATIENT = Column(String, ForeignKey('patients.Id'))
    ORGANIZATION = Column(String)
    PAYER = Column(String, ForeignKey('payers.Id'))
    ENCOUNTERCLASS = Column(String)
    CODE = Column(String)
    DESCRIPTION = Column(String)
    BASE_ENCOUNTER_COST = Column(Float)
    TOTAL_CLAIM_COST = Column(Float)
    PAYER_COVERAGE = Column(Float)
    REASONCODE = Column(String)
    REASONDESCRIPTION = Column(String)
    Year_Month = Column(String)


class Patient(Base):
    __tablename__ = 'patients'
    Id = Column(String, primary_key=True)
    BIRTHDATE = Column(Date)
    DEATHDATE = Column(Date)
    PREFIX = Column(String)
    FIRST = Column(String)
    LAST = Column(String)
    SUFFIX = Column(String)
    MAIDEN = Column(String)
    MARITAL = Column(String)
    RACE = Column(String)
    ETHNICITY = Column(String)
    GENDER = Column(String)
    BIRTHPLACE = Column(String)
    ADDRESS = Column(String)
    CITY = Column(String)
    STATE = Column(String)
    COUNTY = Column(String)
    ZIP = Column(String)
    LAT = Column(Float)
    LON = Column(Float)
    Age = Column(Integer)
    Age_Group = Column(String)


class Procedure(Base):
    __tablename__ = 'procedures'
    Id = Column(String, primary_key=True)
    START = Column(Date)
    STOP = Column(Date)
    PATIENT = Column(String, ForeignKey('patients.Id'))
    ENCOUNTER = Column(String, ForeignKey('encounters.Id'))
    CODE = Column(String)
    DESCRIPTION = Column(String)
    BASE_COST = Column(Float)
    REASONCODE = Column(String)
    REASONDESCRIPTION = Column(String)
    Year_Month = Column(String)


class COVID19TestToTreat(Base):
    __tablename__ = 'COVID_19_Test_to_Treat'
    Provider_Name = Column(String, primary_key=True)
    Address1 = Column(String)
    CITY = Column(String)
    STATE = Column(String)
    Zip = Column(String)
    Last_Report_Date = Column(Date)
    Geopoint = Column(String)


class DailyTownCOVID19(Base):
    __tablename__ = 'Daily_Town_COVID19'
    Date = Column(Date, primary_key=True)
    COUNTY = Column(String)
    Town = Column(String)
    covid_cases_cumulative = Column(Integer)
    daily_cases = Column(Integer)
    Year_Month = Column(String)

# Create the database engine
engine = create_engine('sqlite:///health_data.db')

# Create all tables
Base.metadata.create_all(engine)

print("Database and tables have been created.")

Database and tables have been created.


In [3]:
file_paths = {
    'payers': 'Valid Data/payers-valid.csv',
    'encounters': 'Valid Data/encounters-valid.csv',
    'patients': 'Valid Data/patients-valid.csv',
    'procedures': 'Valid Data/procedures-valid.csv',
    'COVID_19_Test_to_Treat': 'Valid Data/COVID_19_Test_to_Treat-valid.csv',
    'Daily_Town_COVID19': 'Valid Data/Daily_Town_COVID19-valid.csv'
}

In [4]:
for name, path in file_paths.items():
    try:
        size_in_bytes = os.path.getsize(path)
        size_in_mb = size_in_bytes / (1024 * 1024)
        print(f"{name}: {size_in_mb:.2f} MB")
    except OSError as e:
        print(f"Could not get size for {path}: {e}")

payers: 0.00 MB
encounters: 5.00 MB
patients: 0.22 MB
procedures: 5.78 MB
COVID_19_Test_to_Treat: 0.02 MB
Daily_Town_COVID19: 1.77 MB


In [5]:
# 1. Calculate total size of all CSVs
total_size_bytes = 0
sizes = {}
for table_name, file_path in file_paths.items():
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path)
        sizes[table_name] = file_size
        total_size_bytes += file_size
    else:
        sizes[table_name] = 0

# 2. Set your max DB size threshold in bytes (10 MB)
MAX_DB_SIZE_BYTES = 9.5 * 1024 * 1024  # 10 MB in bytes

# 3. For each CSV, decide what fraction to load
#    fraction = (MAX_DB_SIZE_BYTES / total_size_bytes) for the entire dataset
#    But you might want additional safety margin or other logic.
#    We'll do a *global fraction* that is the same for each CSV. 
#    (Because if we do it proportionally per file, we need to ensure the sum is 10 MB.)
if total_size_bytes == 0:
    fraction = 1.0
else:
    fraction = min(1.0, MAX_DB_SIZE_BYTES / total_size_bytes)

# 4. Load data into tables using the fraction
for table_name, file_path in file_paths.items():
    if not os.path.exists(file_path) or sizes[table_name] == 0:
        print(f"File {file_path} doesn't exist or is empty.")
        continue

    # Read the CSV
    df = pd.read_csv(file_path)

    # Sample a fraction of rows
    # fraction is the same for all files here
    # If fraction is 1.0 or more, that means we can load everything
    if fraction < 1.0:
        df = df.sample(frac=fraction, random_state=42)  # set a seed for reproducibility

    # Load into the database
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
    print(f"Loaded {len(df)} rows into '{table_name}' (fraction={fraction:.2f}).")

print("Data has been partially loaded into the database.")

Loaded 7 rows into 'payers' (fraction=0.74).
Loaded 13496 rows into 'encounters' (fraction=0.74).
Loaded 722 rows into 'patients' (fraction=0.74).
Loaded 23001 rows into 'procedures' (fraction=0.74).
Loaded 143 rows into 'COVID_19_Test_to_Treat' (fraction=0.74).
Loaded 31752 rows into 'Daily_Town_COVID19' (fraction=0.74).
Data has been partially loaded into the database.
